---
title: "Juliaを用いたランベール問題のソルバーに関するノートブック" \
author: "Naoya Ozaki" \
date: "04 June 2022" \
output: "lambert_problem_via_julia"

---

# Juliaを用いたランベール問題のソルバーに関するノートブック
数値計算分野で用いられるJuliaを用いてランベール問題(Lambert's Problem)の解法を実装してみよう．

# 1. はじめに
ランベール問題とは，二体問題の力学環境下において，初期位置$\boldsymbol{r}_0$と終端位置$\boldsymbol{r}_\textrm{f}$と遷移時間TOF($=t_f-t_0$)が与えられた時に，初期速度$\boldsymbol{v}_0$と終端速度$\boldsymbol{v}_\textrm{f}$を解く問題であり，二点境界値問題の一種である．与えられた$\boldsymbol{r}_0$, $\boldsymbol{r}_\textrm{f}$, TOFを満たす解は複数存在する（＝一意に定まらない）ため，複数の解が出力される必要がある．`pykep`等でランベール問題用ソルバーが提供されているが，ここではゼロから実装することを考える．本実装を進める上で参考となる論文を以下に挙げる．

### 参考文献・論文
1. Dario Izzo, "Revisiting Lambert's problem", Celestial Mechanics and Dynamical Astoronomy, 2015, Vol 121, pp 1-15.
2. R. H. Battin, "An Introduction to the Mathematics and Methods of Astrodynamics, Revised Version", 1999, AIAA Education Series.
3. PyKEP (https://esa.github.io/pykep/) access on Nov.2017.
4. Fortran-Astrodynamics-Toolkit(https://github.com/jacobwilliams/Fortran-Astrodynamics-Toolkit) access on Nov.2017.

### Julia用のNAIF SPICE Toolkitのインストール
以下のGithubからインストールすることが可能である．\
https://github.com/JuliaAstro/SPICE.jl

参考情報：\
Acton, C.H.; "Ancillary Data Services of NASA's Navigation and Ancillary Information Facility;" Planetary and Space Science, Vol. 44, No. 1, pp. 65-70, 1996.

In [1]:
import Pkg; Pkg.add("SPICE")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [2]:
import SPICE

# Load generic kernels
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/lsk/naif0012.tls") # Leap seconds kernel
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/pck/gm_de431.tpc") # Gravity Constant
SPICE.furnsh("/Users/naoya/Google Drive/21_Programming/SPICE/generic_kernels/spk/planets/de440.bsp") # Planetary ephemeris kernel

# Convert the calendar date to ephemeris seconds past J2000
et = SPICE.utc2et("2018-02-06T20:45:00")

# Get the position of Mars at `et` w.r.t. Earth
SPICE.spkpos("mars_barycenter", et, "J2000", "none", "earth")

([-9.561252750118901e7, -2.045743839170729e8, -8.601309297738348e7], 806.0297773638883)

# 2. 理論編
TBW

# 3. 実装編

## 3.1. 前提条件
以下に示す前提条件でランベール問題の実装を進める．

In [3]:
import SPICE
# 重力定数
μ = SPICE.bodvrd("SUN", "GM", 1)[1];

# 飛行時間
et1 = SPICE.str2et("2031/03/01 00:00:00 UTC") # 地球出発日時, UTC
et2 = SPICE.str2et("2032/01/01 00:00:00 UTC") # 火星到着日時, UTC
tof = et2 - et1 # 飛行時間

#　地球・火星軌道
state_earth, _ = SPICE.spkez(399, et1, "ECLIPJ2000", "NONE", 10)
state_mars, _ = SPICE.spkez(4, et2, "ECLIPJ2000", "NONE", 10)

r1 = state_earth[1:3]
r2 = state_mars[1:3]

3-element Vector{Float64}:
  2.0803122655020934e8
  1.8008145720647745e7
 -4.721309430616008e6

## 3.2. ランベール問題のソルバー
ランベール問題を解く際に利用される変数
$$
\begin{align}
c &= \|\boldsymbol{r}_2 - \boldsymbol{r}_1\| \\
s &= \frac{1}{2}\left(r_1 + r_2 + c\right) \\
\lambda^2 &= 1 - \frac{c}{s}
\end{align}
$$
および軌道を規定する基底を計算する．


In [4]:
using LinearAlgebra

# ランベール問題で用いられる幾何学的な変数
r1_norm = norm(r1)
r2_norm = norm(r2)
c = norm(r2-r1)
s = 0.5*(r1_norm + r2_norm + c)
λ2 = 1.0 - c/s

# 無次元化時刻の計算
tof_nomdim = sqrt(2.0*μ/(s^3))*tof

2.054755701462335

In [5]:
# 基底ベクトル
ivec_r1 = r1/r1_norm
ivec_r2 = r2/r2_norm
ivec_h = cross(ivec_r1, ivec_r2)
ivec_h = ivec_h/norm(ivec_h) # 角運動量ベクトル
# NOTE: r1ベクトルとr2ベクトルが平行な場合，ivec_hが定義できないため，注意が必要．

if ivec_h[3] < 0.0
    λ = -sqrt(λ2)
    ivec_t1 = -cross(ivec_h, ivec_r1)
    ivec_t2 = -cross(ivec_h, ivec_r2)
else
    λ = sqrt(λ2)
    ivec_t1 = cross(ivec_h, ivec_r1)
    ivec_t2 = cross(ivec_h, ivec_r2)
end

3-element Vector{Float64}:
 -0.0872249314162494
  0.9950288911397668
 -0.04805535492114194

In [6]:
function x2tof(x, λ, m_max, Δx_battin=0.01, Δx_lagrange=0.2)
    """ A Method that Calculates tof from x
    """
    Δx = abs(x - 1.0)
    
    if (Δx > Δx_battin) && (Δx < Δx_lagrange)
        # Use Lagrange TOF Expression
        a = 1.0 / (1.0 - x^2) # Non dimensional semi-major axis
        if a > 0 # Ellipse
            α = 2.0*acos(x)
            β = 2.0*asin(λ/sqrt(a))
            tof = 0.5 * a^(3/2)*( (α-sin(α)) - (β-sin(β)) + 2.0*m_max*π ) # Non dimensinal TOF

        else # Hyperbola
            α = 2.0*acosh(x)
            β = 2.0*asinh(λ/sqrt(a))
            tof = - 0.5 * (-a)^(3/2)*( (α-sinh(α)) - (β-sinh(β)) ) # Non dimensinal TOF

        end

    elseif (Δx < Δx_battin)
        # Use Battin Series TOF Expression
        ρ = abs(x^2 - 1.0)
        y = sqrt(1.0 + λ^2*(x^2 - 1.0))
        η = y - λ*x
        s1 = 0.5*(1.0 - λ - x*η)
        q = 4.0/3.0*hypergeometric_₂F₁(s1)
        tof = (η^3*q + 4.0*λ*η)/2.0 + m_max*π/(ρ^1.5)

    else
        # Use Lancaster TOF Expression
        e = x^2 - 1.0
        ρ = sqrt(abs(e))
        y = sqrt(1.0 + λ^2*e)
        if (e<0.0)
            d = m_max*π + acos(x*y - λ*e)
        else
            d = log(ρ*(y-λ*x) + (x*y-λ*e))
        end
        tof = (x - λ*y - d/ρ)/e

    end 

    return tof
end

function hypergeometric_₂F₁(z,a=3.0,b=1.0,c=2.5,tol=1.0e-11)
    # Initilization
    sj, cj, sj1, cj1, err = 1.0, 1.0, 0.0, 0.0, 1.0

    # Iteration
    j = 0
    while (err > tol) 
        cj1 = cj * (a+j)*(b+j)/(c+j)*z/(j+1)
        sj1 = sj + cj1
        err = abs(cj1)
        sj, cj = sj1, cj1
        j += 1
        if j > 1000
            println("ERROR: Hypergeometric Function Reaches Maximum Iteration.")
            break
        end
    end

    return sj
end

hypergeometric_₂F₁ (generic function with 5 methods)

### x, yの解法
$\lambda$と$T$を入力として，$x, y$を計算する．


In [7]:
function find_xy(λ, tof, m_multi_revs)
    # Requirements
    if abs(λ) >= 1
        error("ERROR: Lambda must be more than 1.")
    end
    if tof < 0
        error("ERROR: Non dimensional tof must be a positive number.")
    end

    # ----------------
    # 1. Detect m_max
    m_max = floor(tof/π)
    t_00 = acos(λ) + λ*sqrt(1-λ^2) # Eq.(19) in Ref[1]
    t_0m = t_00 + m_max*π # Minimum Energy Transfer Time: Eq.(19) in Ref[1]
    t_1 = 2.0/3.0*(1.0 - λ^3)

    if (m_max > 0) && (tof < t_0m)
        x_tmin, t_min = find_tof_min_by_halley_method(0.0,t_0m,λ,m_max)

        if tof < t_min
            m_max -= 1
        end
    end

    # Crop m_max to m_multi_revs
    m_max = Int8(min(m_multi_revs, m_max))

    # ----------------
    # 2. Calculate All Solutions in x,y
    x_all = zeros(2*m_max+1)
    iter_all = zeros(Int16,2*m_max+1)

    # 2.1. Single Revolution Solution
    # Initial guess
    if tof >= t_00
        x_all[1] = (t_00/tof)^(2/3) - 1.0
    elseif tof <= t_1
        x_all[1] =  (5.0*t_1*(t_1-tof))/(2.0*tof*(1-λ^5)) + 1.0
    else
        x_all[1] = (t_00/tof)^(log2(t_1/t_00)) - 1.0
    end

    # Householder iterations
    iter_all[1], x_all[1] = find_x_by_householder(tof,x_all[1],λ,0,1.0e-5)

    # 2.2. Multi Revolution Solution
    for i=1:m_max
        # Left Householder iterations
        temp = ((i*π+π)/(8.0*tof))^(2.0/3.0)
        x_all[2*i] = (temp-1.0)/(temp+1.0)
        iter_all[2*i], x_all[2*i] = find_x_by_householder(tof,x_all[2*i],λ,i)

        # Right Householder iterations
        temp = ((8.0*tof)/(i*π))^(2.0/3.0)
        x_all[2*i+1] = (temp-1.0)/(temp+1.0)
        iter_all[2*i+1], x_all[2*i+1] = find_x_by_householder(tof,x_all[2*i+1],λ,i)
    end
    
    return iter_all, x_all
end


function find_x_by_householder(tof, xn, λ, m, tol_Δx=1.0e-8, max_iter=15)
    """
    Find x that satisfies f(x)=tn(x)-tof=0 by Householder's method
    """
    iter = 0
    while true
        tn = x2tof(xn,λ,m)
        yn = sqrt(1.0 - λ^2*(1.0-xn^2))
        # Eqs.(22) in Ref[1]
        f(x,t) = t-tof
        df_dx(x,t) = (3.0*t*x - 2.0 + 2.0*λ^3*x/yn)/(1.0-x^2)
        d2f_dx2(x,t) = (3.0*t + 5.0*x*df_dx(x,t) + 2.0*(1.0-λ^2)*(λ^3)/(yn^3))/(1.0-x^2)
        d3f_dx3(x,t) = (7.0*x*d2f_dx2(x,t) + 8.0*df_dx(x,t) - 6.0*(1.0-λ^2)*(λ^5)*x/(yn^5))/(1.0-x^2)

        # Householder's Method
        xn_new = xn - f(xn,tn)*( 
            (df_dx(xn,tn)^2 - 0.5*f(xn,tn)*d2f_dx2(xn,tn))
            / (df_dx(xn,tn)*(df_dx(xn,tn)^2 - f(xn,tn)*d2f_dx2(xn,tn)) + d3f_dx3(xn,tn)*(f(xn,tn)^2)/6.0 )
        ) 

        # Break condition
        if abs(xn_new - xn) < tol_Δx
            tn = x2tof(xn_new,λ,m)
            return iter, xn_new
        elseif iter > max_iter
            println("Householder iteration reaches maximum iteration!")
            tn = x2tof(xn_new,λ,m)
            return iter, xn_new
        end

        # Update the value
        xn = xn_new
        iter += 1

    end

end


function find_tof_min_by_halley_method(xn, tn, λ, m_max, tol_Δx = 1.0e-13, max_iter=12)
    """
    Find minimum value of transfer time by Halley's method
    """
    iter = 0
    while true
        yn = sqrt(1.0 - λ^2*(1.0-xn^2))
        # Eqs.(22) in Ref[1]
        dt_dx(x,t) = (3.0*t*x - 2.0 + 2.0*λ^3*x/yn)/(1.0-x^2)
        d2t_dx2(x,t) = (3.0*t + 5.0*x*dt_dx(x,t) + 2.0*(1.0-λ^2)*(λ^3)/(yn^3))/(1.0-x^2)
        d3t_dx3(x,t) = (7.0*x*d2t_dx2(x,t) + 8.0*dt_dx(x,t) - 6.0*(1.0-λ^2)*(λ^5)*x/(yn^5))/(1.0-x^2)

        # Halley's Method
        xn_new = xn - (2.0*dt_dx(xn,tn)*d2t_dx2(xn,tn))/(2.0*(d2t_dx2(xn,tn)^2) - dt_dx(xn,tn)*d3t_dx3(xn,tn))

        # Break condition
        if abs(xn_new - xn) < tol_Δx
            tn = x2tof(xn_new,λ,m_max)
            return xn_new, tn
        elseif iter > max_iter
            println("Halley iteration reaches maximum iteration!")
            tn = x2tof(xn_new,λ,m_max)
            return xn_new, tn
        end

        # Update the value
        tn = x2tof(xn_new, λ, m_max)
        xn = xn_new
        iter += 1

    end
end


find_tof_min_by_halley_method (generic function with 3 methods)

### 出発・到着時の速度を計算

In [8]:
# Calculate x
iter_all, x_all = find_xy(λ,tof_nomdim,2)

# Calculate Terminal Velocities
vvec_1_all = zeros(size(x_all)[1],3)
vvec_2_all = zeros(size(x_all)[1],3)

γ = sqrt(0.5*μ*s)
ρ = (r1_norm - r2_norm)/c
σ = sqrt(1.0-ρ^2)

for i = 1:(size(x_all)[1])
    # Calculate Velocity Norm
    x = x_all[i]
    y = sqrt(1.0 - λ^2*(1.0-x^2))
    v_r1 = γ*((λ*y-x) - ρ*(λ*y+x))/r1_norm
    v_r2 = -γ*((λ*y-x) + ρ*(λ*y+x))/r2_norm
    v_t1 = γ*σ*(y+λ*x)/r1_norm
    v_t2 = γ*σ*(y+λ*x)/r2_norm

    # Calculate Velocity Vectors
    vvec_1_all[i,:] = v_r1*ivec_r1 + v_t1*ivec_t1
    vvec_2_all[i,:] = v_r2*ivec_r2 + v_t2*ivec_t2
end

vvec_1_all, vvec_2_all

([-12.146901903784702 -30.249196666164888 1.7328230818673045], [-5.062557143526499 22.77374756805452 -1.0435265077509397])